# Script to automate the export and manipulation of the VICAV-library

## Import Package eTree to parse XML Files

In [1]:
import requests
import json
import logging
import os
import xml
import xml.etree.ElementTree as ET
import asyncio
import aiohttp
# this module is needed to make asyncio.run work inside the notebook as well as in the generated python script
import nest_asyncio
nest_asyncio.apply()
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)
#logging.basicConfig(level=logging.DEBUG)

## Define name-space for xml-parsing

In [2]:
xmlns = {"tei": "http://www.tei-c.org/ns/1.0", "xml":"http://www.w3.org/XML/1998/namespace" }

## Access to the VICAV Zotero library

* Use API_TOKEN from environment to access Zotero
* Set the Zotero group id for VICAV here

In [3]:
request_headers = {'Authorization': 'Bearer ' + os.environ['API_TOKEN']}
group_id = "2165756"
limit_downloads_to = None
conn_limit=16
total_timeout=600 #s

## Read all items in the library

Load items from Zotero group library

    Args: 
        group_id (str): ID of a Zotero group
        limit (int): number of items to retrieve from library, maximum is 100.
        start (int): item number to start with

In [4]:
async def get_items(session, group_id:str,limit:int,start:int,itemType = None):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + "?limit=" + str(limit) + "&start=" + str(start) + ("&itemType="+itemType if itemType is not None else "")
    async with session.get(url=request_url, headers=request_headers) as response:
        if response.status == 200:
            parsed = json.loads(await response.text())
            response_headers = response.headers
        
    return parsed, response_headers

Get total number of items in group library

    Args:  
        group_id (str): ID of a Zotero group
    
    Returns:
        int: number of items in the library

In [5]:
def total_number_items(group_id) -> int:
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/"
    response = requests.get(request_url, headers=request_headers)
    
    return int(response.headers["Total-Results"])

Get headers of Zotero-Api-Calls

    Args:  
        group_id (str): ID of a Zotero group

In [6]:
def get_headers(group_id):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/"
    response = requests.get(request_url, headers=request_headers)
    
    return response.headers

Get links from headers

    Args:
        headers: http-headers of a response

    Returns:
        dict

In [7]:
def get_links_from_headers(headers) -> dict:
    link_list = headers["Link"].split(",")
    links = {}
    for link_item in link_list:
        #print(link_item)
        link_type = link_item.split('; rel="')[1].replace('"','').strip()
        link_value = link_item.split('; rel="')[0].replace("<","").replace(">","").strip()
        links[link_type] = link_value
    
    return links

Get all items of a collection

In [8]:
async def get_all_items(session, group_id, itemType = None):
    logging.info("Getting all items" + ((" of type " + itemType) if itemType is not None else "") + " now.")
    # empty list that will hold all items of the library
    allitems=[]
    
    # settings to be used in the function to get the items (limit is max 100)
    limit=100
    start=0
    
    # get the first 200 items to start with
    first_round=await get_items(session, group_id,limit,start,itemType)
    allitems=allitems+first_round[0]
    
    # get the next link from the headers
    next_url = get_links_from_headers(first_round[1])["next"]
    last_url = get_links_from_headers(first_round[1])["last"]
    # get items until next url is last url, then all items are fetched
    while next_url != last_url:
        logging.info("Getting items from " + next_url)
        async with session.get(url=next_url, headers=request_headers) as response:
            if response.status == 200:
                parsed = json.loads(await response.text())
                response_headers = response.headers

                allitems=allitems + parsed
                urls = get_links_from_headers(response_headers)
                if "next" in urls:
                    next_url = urls["next"]
                else:
                    break
            else:
                break

    # get the last items of the group
    response = requests.get(last_url)
    if response.status_code == 200:
        parsed = json.loads(response.text)
        allitems=allitems + parsed
    
    return allitems

Store all items of a group library in a json file

    Args:
        group_id (str): ID of a Zotero group
        filename (str): name of the export file including file-extension

    Returns:
        bool: True if successful

In [9]:
def export_all_items_to_file(group_id,filename) ->bool: 
    allitems = get_all_items(group_id)
    with open(filename,"w") as f:
        json.dump(allitems, f)
    return True

Store export in a file and get all item ids

The export contains also the note items. These are child items of some other item in this export. They have a parent reference.

In [10]:
json_file = "export_grouplib.json"
item_ids = []
note_ids = []
async def get_generic_items(session):
    if os.path.isfile(json_file):
        logging.info("Grouplib export json already exists. Delete to fetch again (time consuming).")
        with open(json_file, 'r') as f:
            all_items = json.load(f)    
    else:
        all_items = await get_all_items(session, group_id)
    return all_items

async def get_export_json():
    conn = aiohttp.TCPConnector(limit=conn_limit)
    timeout = aiohttp.ClientTimeout(total=total_timeout)
    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        all_items = await get_generic_items(session)
    # all_items = test5
    with open(json_file,"w") as f:
        json.dump(all_items, f)
        logging.info("Exported json.")
    
    for item in all_items:
        item_id = item["key"]
        item_type = item["data"]["itemType"]
        if item_type == 'note':
            note_ids.append(item_id)
        else:
            item_ids.append(item_id)
    return all_items
all_items = asyncio.run(get_export_json())
all_items_map = {data["key"]:data for data in all_items}
all_notes_map = {data["data"]["parentItem"]:data for data in [all_items_map[id] for id in note_ids]}

2023-02-22 21:22:07,465 - Grouplib export json already exists. Delete to fetch again (time consuming).
2023-02-22 21:22:08,857 - Exported json.


## Get all TEIs from Zotero

man nimmt die Liste mit den IDs der entries, baut für jeden entry die URL nach dem Muster  
https://api.zotero.org/groups/2165756/items/944KQVKQ?format=tei  
man lädt das mit GET requesst  
dann aus dem response den body und parsed das mit ET from string, nimmt daraus das  
`<biblStruct>` Element;  
baut eine gemeinsame `<listBibl>` und fügt das geparste Element ein,  
dann dumpt man den ganzen Element-Tree

### Retrieves TEI of an item generated by Zotero

In [43]:
ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
async def get_item_tei(group_id,item_id,session):
    request_url = "https://api.zotero.org/groups/" + group_id + "/items/" + item_id + "?format=tei"
    bibl = None
    note_for_item_id = ""
    response_text = ""
    try:
        async with session.get(url=request_url, headers=request_headers) as response:
            response_text = await response.text()
        list_bibl = ET.fromstring(response_text)
        bibl = list_bibl.find("tei:biblStruct",xmlns)
        if bibl is None:
            logging.info("There is no biblStruct in the response:\n"+response_text)
            return bibl
        if item_id in all_notes_map:
            note_for_item_id = all_notes_map[item_id]["data"]["note"].replace("&", "&amp;")
            parsed_note = ET.fromstring("<note>"+note_for_item_id+"</note>")
            bibl.append(parsed_note)
        tags_el = ET.SubElement(bibl, "note", type="tags")
        for o in all_items_map[item_id]["data"]["tags"]:
            ET.SubElement(tags_el, "note", type="tag").text = o["tag"]
    except asyncio.TimeoutError:
        logging.info("Timeout fetching " + item_id)
    except xml.etree.ElementTree.ParseError:
        logging.info("XML parser error in notes for item id "+item_id+"\n"+note_for_item_id)
    if bibl is None:
        logging.debug("No biblStruct in item " + item_id)
    logging.info("Fetched TEI for " + item_id)
    return bibl

In [44]:
async def get_item_tei_test():
    conn = aiohttp.TCPConnector(limit=conn_limit)
    timeout = aiohttp.ClientTimeout(total=total_timeout)
    async with aiohttp.ClientSession(connector=conn) as session:
        #test = await get_item_tei(group_id,"944KQVKQ",session)
        #test = await get_item_tei(group_id,"DXNCFAMR",session)
        test = await get_item_tei(group_id,"6QNLQCG9",session)
    ET.indent(test)
    ET.dump(test)
asyncio.run(get_item_tei_test())

2023-02-22 21:50:54,958 - Fetched TEI for 6QNLQCG9


<biblStruct xmlns="http://www.tei-c.org/ns/1.0" type="book" xml:id="Alhawary2018" corresp="http://zotero.org/groups/2165756/items/6QNLQCG9">
  <monogr>
    <title level="m">The Routledge handbook of Arabic second language acquisition</title>
    <idno type="ISBN">978-1-138-94055-0</idno>
    <idno type="callNumber">PJ6065 .R68 2018</idno>
    <editor>
      <forename>Mohammad T.</forename>
      <surname>Alhawary</surname>
    </editor>
    <imprint>
      <pubPlace>Milton Park, Abingdon, Oxon</pubPlace>
      <publisher>New York, NY : Routledge/ Taylor &amp; Francis Group</publisher>
      <date>2018</date>
    </imprint>
  </monogr>
  <note>Introduction -- Arabic L2 phonology. the perception and production of Arabic lexical stress by L1 English and L1 Chinese learners of Arabic: a usage-based account / Cheng-Wei Lin and Mohammad T. Alhawary -- Production of modern standard Arabic lexical stress cues by native speakers of American English / Mashael Al-Aloula -- Native English speakers

In [45]:
print("Notes:\n", all_notes_map["944KQVKQ"]["data"]["note"], "\nTags:\n", all_items_map["944KQVKQ"]["data"]["tags"])

Notes:
 <p>Accession Number: ICHA1094316. Harahsheh, Ahmad Mohammad Ahmad al-; Rousan, Rafat M. al. Issue Info: 47 i. Publication Date: 20200101. Number of Pages: 9. Document Type: Article. Language: English.</p> 
Tags:
 [{'tag': 'Animals (in Islamic arts, literatures, folklore, traditions, cultures, law)', 'type': 1}, {'tag': 'Anthropology & ethnography', 'type': 1}, {'tag': 'Arabic language: colloquial', 'type': 1}, {'tag': 'Colloquial Arabic dialects Shami', 'type': 1}, {'tag': 'Jordan', 'type': 1}, {'tag': 'Jordan Social studies', 'type': 1}]


### Load template containing a listBibl-element that will be filled with the retrieved biblStruct elements

In [46]:
template = ET.parse("listbibl_template.xml")
list_bibl = template.find("tei:text/tei:body/tei:listBibl",xmlns)

## Get the TEI

* For each item-id get the TEI and append it to list-bibl
* Save the resulting XML
* Save errors for further inspection

We need to consider https://www.zotero.org/support/dev/web_api/v3/basics#rate_limiting

In [47]:
errors = []
max_fetch = limit_downloads_to if limit_downloads_to is not None else len(item_ids)
logging.info("Fetching at most " + str(max_fetch) + " TEI/XML.")
async def get_item_tei_from_group(item_id, session):
    bibl_struct = await get_item_tei(group_id, item_id, session)
    if bibl_struct:
        list_bibl.append(bibl_struct)
    else:
        logging.debug("Can not append " + item_id)
        errors.append(item_id)

async def async_download():
    conn = aiohttp.TCPConnector(limit=conn_limit)
    timeout = aiohttp.ClientTimeout(total=total_timeout)
    async with aiohttp.ClientSession(connector=conn, timeout=timeout) as session:
        await asyncio.gather(*[get_item_tei_from_group(item_id, session) for item_id in item_ids[:max_fetch]])

asyncio.run(async_download())

with open('TEI_export.xml', 'wb') as f:
    ET.indent(template)
    template.write(f, encoding='utf-8')
    logging.info("TEI export done.")

# Export IDs of items with errors
with open("errors.json","w") as f:
    json.dump(errors, f)
    logging.info("Exported errors.json.")

2023-02-22 21:51:09,413 - Fetching at most 5185 TEI/XML.
2023-02-22 21:51:12,569 - Fetched TEI for B6L9AIMZ
2023-02-22 21:51:12,751 - Fetched TEI for NPJ679RJ
2023-02-22 21:51:12,766 - Fetched TEI for QUCSQL79
2023-02-22 21:51:12,773 - Fetched TEI for 6EPNLRXU
2023-02-22 21:51:12,783 - Fetched TEI for 4RQ672D6
2023-02-22 21:51:12,789 - Fetched TEI for EYRYK2V5
2023-02-22 21:51:12,799 - Fetched TEI for RCI9I9AM
2023-02-22 21:51:12,803 - Fetched TEI for JYWMA77R
2023-02-22 21:51:12,811 - Fetched TEI for N2NCTC8C
2023-02-22 21:51:12,820 - Fetched TEI for 5AGSDUQK
2023-02-22 21:51:12,822 - Fetched TEI for ZPP5LFIZ
2023-02-22 21:51:12,829 - Fetched TEI for RUFEDDWU
2023-02-22 21:51:12,840 - Fetched TEI for BBB3B6VY
2023-02-22 21:51:12,845 - Fetched TEI for 6KWTRZHW
2023-02-22 21:51:12,983 - Fetched TEI for E9GJGZZ7
2023-02-22 21:51:12,998 - Fetched TEI for RNJVZ5ZY
2023-02-22 21:51:13,052 - Fetched TEI for NIGWMNVB
2023-02-22 21:51:13,206 - Fetched TEI for PYD8MYVU
2023-02-22 21:51:13,209 -

KeyboardInterrupt: 

## Generate note tags for tags for each tag from group lib export

Is this the way to go?